In [3]:
# enumerate dates from 2008

import datetime

def yield_datetime(start: datetime.datetime, end: datetime.datetime = None):
    # endless loop if end is not specified
    if end is None:    
        while True:
            yield start
            start += datetime.timedelta(days=1)
    else:
        while start < end:
            yield start
            start += datetime.timedelta(days=1)

In [4]:
!pip install warcio
import warcio
?warcio.WARCWriter
writer = warcio.WARCWriter(None)
?writer.create_warc_record

Signature:
writer.create_warc_record(
    uri,
    record_type,
    payload=None,
    length=None,
    warc_content_type='',
    warc_headers_dict=None,
    warc_headers=None,
    http_headers=None,
)
Docstring: <no docstring>
File:      /workspaces/handy-snippets-in-jupyter-notebook/.venv/lib/python3.12/site-packages/warcio/recordbuilder.py
Type:      method

In [5]:
!pip install pytest
# create a block except exception to raise TypeError    
import pytest

with pytest.raises(TypeError):
    raise TypeError


In [6]:
!pip install requests
import requests

# read all dates like https://unwire.hk/2025/08/21/

# save them as common crawl format (see https://commoncrawl.org/the-data/)
import pytest

!pip install tqdm
from tqdm.auto import tqdm
with pytest.raises(TypeError):
    for date in tqdm(list(yield_datetime(datetime.datetime(2009, 1, 7), end=datetime.datetime.today()))):
        url = f"https://unwire.hk/{date:%Y/%m/%d}/"
        r = requests.get(url)
        if r.status_code == 200:
            # AttributeError: module 'warcio' has no attribute 'WarcFile'
            with open("unwire.hk.warc", "ab") as f:
                writer = warcio.WARCWriter(f)
                writer.create_warc_record(uri=url)

  0%|          | 0/6071 [00:00<?, ?it/s]


In [7]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content, "html.parser")

In [8]:
!pip install pandas
import pandas as pd
alist = pd.Series(soup.find_all("a"))
alist

0                                [[], []]
1                               [[Learn]]
2                         [GenAI 與雲端科技專區]
3                         [ERP 與商業 AI 專區]
4                       [Galaxy Z Fold 7]
                      ...                
103                              [作者忘記分類]
104    [HP Pavilion dv2－首部 AMD Neo 平台現身！]
105                                  [[]]
106                                  [[]]
107                                  [[]]
Length: 108, dtype: object

In [9]:
alist.apply(lambda x: x.get("href"))

0                                     https://unwire.hk/
1                                 https://list.unwire.hk
2                     https://unwire.hk/tag/genai-cloud/
3                 https://unwire.hk/tag/erp-business-ai/
4                 https://unwire.hk/tag/galaxy-z-fold-7/
                             ...                        
103                     https://unwire.hk/category/news/
104    https://unwire.hk/2009/01/07/hp-pavilion-dv2%e...
105    https://www.facebook.com/sharer.php?u=https://...
106    https://twitter.com/intent/tweet?url=https://u...
107                                                _self
Length: 108, dtype: object

In [10]:
import bs4
def get_css_selector_for_element(element: bs4.element.Tag):
    # get ancestors
    ancestors = []
    while element is not None:
        # use id, class-name, tag-name in order
        if element.has_attr("id"):
            ancestors.append(f"#{element['id']}")
            # id is unique, so stop
            break
        elif element.has_attr("class"):
            ancestors.append(f".{element['class'][0]}")
        else:
            ancestors.append(element.name)
        element = element.parent
    ancestors.reverse()
    del ancestors[0]
    # print(ancestors)
    # get css selector
    css_selector = " > ".join(ancestors)
    return css_selector

In [11]:
alist.apply(get_css_selector_for_element).value_counts()
# right one is #archive-main-content > div

.no-js > .archive > .site-header > .top-bar-tags > .top-bar-tags-container > ul > li > a                                                            43
.menu-section > ul > li > li > li > .sub-list > ul > li > span > a                                                                                  12
.menu-section > ul > li > li > .sub-list > ul > li > span > a                                                                                        8
.menu-section > ul > li > .sub-list > ul > li > span > a                                                                                             7
.menu-section > ul > li > li > li > .first-level > a                                                                                                 5
.main-content-container > .article-section > .articles-container > ul > li > .post-content > .post-meta > .post-meta-share > ul > li > .twitter      4
.main-content-container > .article-section > .articles-container > ul > li > .post-content > .

In [12]:
url

'https://unwire.hk/2009/01/07/'

In [13]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Firefox()
driver.get(url)
time.sleep(3)
html = driver.page_source
driver.quit()   

WebDriverException: Message: Process unexpectedly closed with status 255
